In [53]:
import copick

validate_runs = "TS_86_3"
config = "train_copick.config"
copickRoot = copick.from_file(config)
run_ids = [run.name for run in copickRoot.runs if run.name != validate_runs]

In [54]:
run_ids


['TS_5_4', 'TS_69_2', 'TS_6_4', 'TS_6_6', 'TS_73_6', 'TS_99_9']

In [4]:
def get_pickable_object_label(copickRoot, objectName):
    for ii in range(len(copickRoot.pickable_objects)):
        if copickRoot.pickable_objects[ii].name == objectName:
            return copickRoot.pickable_objects[ii].label

## How to read and write using CoPick

In [14]:
seg= run.get_segmentations(name="remotetargets",user_id="deepfindET",session_id="0")[0]
store = seg.zarr()
target = zarr.open(store, mode="r")[0][:]

## Group
tomogram = run.get_voxel_spacing(10).get_tomograms("denoised")[0]
data = zarr.open(tomogram.zarr())[0][:]
#query = [] 
#labelList = []
#tomoIDList = []
#pickIndList = []
#proteinIndList = []
#proteinCoordsList = []


#for proteinInd in range(len(query)):
#    picks = query[proteinInd]
#    npicks = len(picks.points)
#    tomoIDList.append(validate_runs * npicks)
#    pickIndList.append(list(range(npicks)))
#    proteinIndList.append([proteinInd] * npicks)
#    proteinCoordsList.append(picks.points)
#    labelList.append([get_pickable_object_label(copickRoot, picks.pickable_object_name)] * npicks)
#labelList = np.array(list(chain.from_iterable(labelList)))
#tomoIDList = np.array(list(chain.from_iterable(tomoIDList)))
#pickIndList = np.array(list(chain.from_iterable(pickIndList)))
#proteinIndList = np.array(list(chain.from_iterable(proteinIndList)))
#proteinCoordsList = np.array(list(chain.from_iterable(proteinCoordsList)))


In [7]:
import zarr
import numpy as np
from itertools import chain
import pandas as pd
copick_path="train_copick.config"
copickRoot = copick.from_file(copick_path)

targets = [obj.name for obj in copickRoot.pickable_objects if obj.is_particle]

validate_runs = "TS_86_3"
run = copickRoot.get_run(validate_runs)

query = []
all_rows = []
pick_id = 0
for target in targets:
    picks = run.get_picks(object_name=target)
    picks = picks[0]
    for point in picks.points:
        
        row = {"id": pick_id, "experiment": validate_runs, "particle_type": target, "x": point.location.x, "y": point.location.y, "z": point.location.z}
        all_rows.append(row)
        pick_id = pick_id + 1

df_reference = pd.DataFrame(all_rows)

In [16]:
data.shape

(184, 630, 630)

### Generate Random Proteins

In [49]:
import numpy as np
validate_runs = "TS_86_3"
run = copickRoot.get_run(validate_runs)
voxel_size = 10 
prediction_rows = []
pick_id = 0
num_obj_objects = np.random.randint(2250)    
for i in range(num_obj_objects):
    target = np.random.choice(["beta-galactosidase", "virus-like-particle"])
    z = np.random.rand(1)[0] * data.shape[0] * voxel_size 
    
    y = np.random.rand(1)[0] *  data.shape[1] * voxel_size
    x = np.random.rand(1)[0] * data.shape[2] * voxel_size
    row = {"id": pick_id, "experiment": validate_runs, "particle_type": target, "x": x, "y": y, "z": z}
    prediction_rows.append(row)
    pick_id = pick_id + 1

In [50]:
df_prediction = pd.DataFrame(prediction_rows)

In [51]:
df_prediction

,id,experiment,particle_type,x,y,z
0,0,TS_86_3,virus-like-particle,3554.258369,5002.427366,968.039778
1,1,TS_86_3,beta-galactosidase,5897.360884,1724.421942,1786.022687
2,2,TS_86_3,beta-galactosidase,5444.986867,2213.417297,1531.682328
3,3,TS_86_3,virus-like-particle,6255.125633,6087.328800,225.339795
4,4,TS_86_3,beta-galactosidase,612.861073,2003.640296,1074.791212
...,...,...,...,...,...,...
875,875,TS_86_3,virus-like-particle,5459.261550,4796.299374,638.185846
876,876,TS_86_3,virus-like-particle,3264.554799,3159.398348,1220.229913
877,877,TS_86_3,virus-like-particle,1776.092096,160.500793,1124.455220
878,878,TS_86_3,virus-like-particle,1981.911366,1533.013188,1804.638784


In [33]:
df_reference

,id,experiment,particle_type,x,y,z
0,0,TS_86_3,apo-ferritin,3870.343,4952.714,1261.600
1,1,TS_86_3,apo-ferritin,4130.897,5422.292,501.860
2,2,TS_86_3,apo-ferritin,2735.000,4668.447,520.291
3,3,TS_86_3,apo-ferritin,2649.615,4690.615,600.923
4,4,TS_86_3,apo-ferritin,2665.353,4810.641,612.019
...,...,...,...,...,...,...
220,220,TS_86_3,virus-like-particle,4683.103,1546.998,899.626
221,221,TS_86_3,virus-like-particle,3563.170,2650.076,660.386
222,222,TS_86_3,virus-like-particle,3994.606,2797.533,839.291
223,223,TS_86_3,virus-like-particle,3829.926,2129.623,859.549


In [47]:
"""
Derived from:
https://github.com/cellcanvas/album-catalog/blob/main/solutions/copick/compare-picks/solution.py
"""

import numpy as np
import pandas as pd

from scipy.spatial import KDTree


class ParticipantVisibleError(Exception):
    pass


def compute_metrics(reference_points, reference_radius, candidate_points):
    num_reference_particles = len(reference_points)
    num_candidate_particles = len(candidate_points)

    if len(reference_points) == 0:
        return 0, num_candidate_particles, 0

    if len(candidate_points) == 0:
        return 0, 0, num_reference_particles

    ref_tree = KDTree(reference_points)
    candidate_tree = KDTree(candidate_points)
    raw_matches = candidate_tree.query_ball_tree(ref_tree, r=reference_radius)
    matches_within_threshold = []
    for match in raw_matches:
        matches_within_threshold.extend(match)
    # Prevent submitting multiple matches per particle.
    # This won't be be strictly correct in the (extremely rare) case where true particles
    # are very close to each other.
    matches_within_threshold = set(matches_within_threshold)
    tp = int(len(matches_within_threshold))
    fp = int(num_candidate_particles - tp)
    fn = int(num_reference_particles - tp)
    return tp, fp, fn


def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        row_id_column_name: str,
        distance_multiplier: float,
        beta: int) -> float:
    '''
    F_beta
      - a true positive occurs when
         - (a) the predicted location is within a threshold of the particle radius, and
         - (b) the correct `particle_type` is specified
      - raw results (TP, FP, FN) are aggregated across all experiments for each particle type
      - f_beta is calculated for each particle type
      - individual f_beta scores are weighted by particle type for final score
    '''

    particle_radius = {
        'apo-ferritin': 60,
        'beta-amylase': 65,
        'beta-galactosidase': 90,
        'ribosome': 150,
        'thyroglobulin': 130,
        'virus-like-particle': 135,
    }

    weights = {
        'apo-ferritin': 1,
        'beta-amylase': 0,
        'beta-galactosidase': 2,
        'ribosome': 1,
        'thyroglobulin': 2,
        'virus-like-particle': 1,
    }

    particle_radius = {k: v * distance_multiplier for k, v in particle_radius.items()}

    # Filter submission to only contain experiments found in the solution split
    split_experiments = set(solution['experiment'].unique())
    submission = submission.loc[submission['experiment'].isin(split_experiments)]

    # Only allow known particle types
    if not set(submission['particle_type'].unique()).issubset(set(weights.keys())):
        raise ParticipantVisibleError('Unrecognized `particle_type`.')

    assert solution.duplicated(subset=['experiment', 'x', 'y', 'z']).sum() == 0
    assert particle_radius.keys() == weights.keys()

    results = {}
    for particle_type in solution['particle_type'].unique():
        results[particle_type] = {
            'total_tp': 0,
            'total_fp': 0,
            'total_fn': 0,
        }

    for experiment in split_experiments:
        for particle_type in solution['particle_type'].unique():
            reference_radius = particle_radius[particle_type]
            select = (solution['experiment'] == experiment) & (solution['particle_type'] == particle_type)
            reference_points = solution.loc[select, ['x', 'y', 'z']].values

            select = (submission['experiment'] == experiment) & (submission['particle_type'] == particle_type)
            candidate_points = submission.loc[select, ['x', 'y', 'z']].values

            if len(reference_points) == 0:
                reference_points = np.array([])
                reference_radius = 1

            if len(candidate_points) == 0:
                candidate_points = np.array([])

            tp, fp, fn = compute_metrics(reference_points, reference_radius, candidate_points)

            results[particle_type]['total_tp'] += tp
            results[particle_type]['total_fp'] += fp
            results[particle_type]['total_fn'] += fn

    aggregate_fbeta = 0.0
    for particle_type, totals in results.items():
        tp = totals['total_tp']
        fp = totals['total_fp']
        fn = totals['total_fn']

        precision = tp / (tp + fp) if tp + fp > 0 else 0
        recall = tp / (tp + fn) if tp + fn > 0 else 0
        fbeta = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall) if (precision + recall) > 0 else 0.0
        aggregate_fbeta += fbeta * weights.get(particle_type, 1.0)

    if weights:
        aggregate_fbeta = aggregate_fbeta / sum(weights.values())
    else:
        aggregate_fbeta = aggregate_fbeta / len(results)
    return aggregate_fbeta



In [52]:
score(df_reference, df_prediction, "id", distance_multiplier=1, beta=4)

0.014060150375939847